In [ ]:
#!pip install selenium
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
from selenium import webdriver
import csv
import os

class CourseraScraper:
    
    def __init__(self,driver,max_pages):
        self.page = 1
        self.max_pages = max_pages
        self.driver =driver
        
    def set_url(self):
        return f"https://fr.coursera.org/search?query=guided%20projects&page={self.page}&index=prod_all_launched_products_term_optimization"
        
    def initialize_headless_browser(self):
        self.driver.get(self.set_url())
        self.driver.implicitly_wait(80)
        return self.driver
    
    def get_content(self):
        self.initialize_headless_browser()
        element = self.driver.find_element_by_class_name('ais-InfiniteHits')
        return  BeautifulSoup(self.driver.page_source, 'html.parser')  #page_content
        
        
    def Scraper(self):
        dataset={}
        
        for page in range(1,self.max_pages+1):
            if self.get_content() is None:
                print("Page not found")
            else:
                courses = self.get_content().find_all("li", {'class': 'ais-InfiniteHits-item'})
                for course in courses:
                    try:
                        course_title = course.h2.get_text()
                        course_rating = course.find('span', {'class': 'ratings-text'}).get_text()
                        course_difficulty = course.find('span', {'class': 'difficulty'}).get_text()
                        if course_difficulty != "Beginner" :
                            dataset.update({course_title: course_rating})
                            print(f"Course Title: \t {course_title}")
                            print(f"Course Rating: \t {course_rating}")
                            print(f"Course Difficulty: \t {course_difficulty}")
                    except:
                        print("Something went wrong when printing the courses")
            self.driver.implicitly_wait(80)
            self.page += 1
            
        self.save(dataset)
        self.quit_headless_browser()
            
    def quit_headless_browser(self):
        self.driver.quit()
        
    def save(self,dataset):
        download_dir = "Coursera"
        if not os.path.exists(download_dir):
            os.mkdir(download_dir)
        filepath = f"{os.path.join(os.path.realpath(os.getcwd()),download_dir,'Coursera')}.csv"
        with open(filepath,"w") as f:
            writer = csv.writer(f)
            writer.writerow(["job title","job rating"])
            for key, value in dataset.items():
                writer.writerow([key, value])
                
        
if __name__ == "__main__":
    path = "C:\Program Files (x86)\chromedriver_win32\chromedriver.exe"
    scraper = CourseraScraper( webdriver.Chrome(path),6)
    scraper.Scraper()